In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
import selenium.webdriver.support.expected_conditions as EC
from tqdm import tqdm_notebook

In [2]:
#  download webpage with sessions first
SessionList = open(r'C:\Users\venug\Downloads\Meeting Library _ 2020 ASCO Virtual Scientific Program.html',encoding = 'utf8')
SessionList = SessionList.read()
SessionList

'<!DOCTYPE html>\n<!-- saved from url=(0099)https://meetinglibrary.asco.org/browse-meetings/2020%20ASCO%20Virtual%20Scientific%20Program?page=1 -->\n<html lang="en" dir="ltr" class=" js"><head class="at-element-marker"><meta http-equiv="Content-Type" content="text/html; charset=UTF-8"><style class="vjs-styles-defaults">\n      .video-js {\n        width: 300px;\n        height: 150px;\n      }\n\n      .vjs-fluid {\n        padding-top: 56.25%\n      }\n    </style>\n    <script type="text/javascript" async="" src="./Meeting Library _ 2020 ASCO Virtual Scientific Program_files/cool-2.1.15.min.js.download"></script><script src="./Meeting Library _ 2020 ASCO Virtual Scientific Program_files/amp4ads-host-v0.js.download"></script><script type="text/javascript" src="./Meeting Library _ 2020 ASCO Virtual Scientific Program_files/a9681c2616"></script><script src="./Meeting Library _ 2020 ASCO Virtual Scientific Program_files/nr-spa-1167.min.js.download"></script><script async="" crossorigin="

In [10]:
SessionData = BeautifulSoup(SessionList,'lxml')
SessionData

<!DOCTYPE html>
<!-- saved from url=(0099)https://meetinglibrary.asco.org/browse-meetings/2020%20ASCO%20Virtual%20Scientific%20Program?page=1 --><html class="js" dir="ltr" lang="en"><head class="at-element-marker"><meta content="text/html; charset=utf-8" http-equiv="Content-Type"/><style class="vjs-styles-defaults">
      .video-js {
        width: 300px;
        height: 150px;
      }

      .vjs-fluid {
        padding-top: 56.25%
      }
    </style>
<script async="" src="./Meeting Library _ 2020 ASCO Virtual Scientific Program_files/cool-2.1.15.min.js.download" type="text/javascript"></script><script src="./Meeting Library _ 2020 ASCO Virtual Scientific Program_files/amp4ads-host-v0.js.download"></script><script src="./Meeting Library _ 2020 ASCO Virtual Scientific Program_files/a9681c2616" type="text/javascript"></script><script src="./Meeting Library _ 2020 ASCO Virtual Scientific Program_files/nr-spa-1167.min.js.download"></script><script async="" crossorigin="anonymous" src="./

In [18]:
SessionLinksData = SessionData.find_all('a',class_ = 'ng-star-inserted')

In [81]:
SessionLinksDup = [i['href'] for i in SessionLinksData ]

In [82]:
SessionLinksDupSet = set(SessionLinksDup)

In [83]:
SessionLinks = []
for link in SessionLinksDup:
    if link in SessionLinksDupSet:
        SessionLinksDupSet.remove(link)
        SessionLinks.append(link)

In [84]:
SessionLinks.remove('https://meetings.asco.org/AM')

In [85]:
SessionLinks.remove('https://rah.asco.org/sso/login.aspx?meetingID=258')

In [86]:
len(SessionLinks)

103

In [87]:
SessionLinks[75]

'https://meetinglibrary.asco.org/session/12674'

In [37]:
driver = webdriver.Chrome(r'C:\Users\venug\Downloads\chromedriver.exe')

In [89]:
driver = webdriver.Chrome(r'C:\Users\venug\Downloads\chromedriver.exe')
dfDetails = pd.DataFrame()
for i in tqdm_notebook(SessionLinks):
    presDetailsDict = {}
    driver.get(i)
    timeout = 500
    locator = '/html/body/div[3]/div/main/div/div/div/ml-root/session/div/div[2]/div/div/div[1]/div[2]/h4'
    WebDriverWait(driver, timeout).until(EC.visibility_of_element_located((By.XPATH, locator)))
    data = driver.page_source
    presList = BeautifulSoup(data,'lxml')
    presentations = presList.find_all('div',class_ = 'session-presentation ng-star-inserted')
    try:
        sessionMainTitle = presList.find('div',class_ = 'title').text
    except:
        sessionMainTitle = ''
    try:
        sessionChairs = presList.find('div',class_ = 'asset-metadata-item session-chair ng-star-inserted').text
    except:
        sessionChairs = ''
    try:
        sessionType = presList.find('div',class_ = 'asset-metadata-item session-type ng-star-inserted').text
    except:
        sessionType = ''
    try:
        sessionTrack = presList.find('div',class_ = 'asset-metadata-item session-tracks ng-star-inserted').text
    except:
        sessionTrack = ''
    if(len(presentations) ==0):
        presDetailsDict = {}
        presDetailsDict['Session Chairs'] = sessionChairs
        presDetailsDict['Session Type']  = sessionType
        presDetailsDict['Session Track'] = sessionTrack
        presDetailsDict['Session Title'] = sessionMainTitle
        dfDetails = dfDetails.append(presDetailsDict,ignore_index = True)

    else:
        for presentation in tqdm_notebook(presentations):
            presDetailsDict = {}
            presDetailsDict['Session Chairs'] = sessionChairs
            presDetailsDict['Session Type']  = sessionType
            presDetailsDict['Session Track'] = sessionTrack
            presDetailsDict['Session Title'] = sessionMainTitle

            try:
                presTitle = presentation.find('div',class_ = 'record__title ng-star-inserted').text
                presDetailsDict['Presentation Title'] = presTitle
            except:
                presTitle = ''
            details = presentation.findAll('div',class_ = 'record__meta ng-star-inserted')
            for i in details:
                try:
                    presDetailsDict[i.find('div',class_ = 'record__meta__title').text] = i.text
                except:
                    pass
            try:
                abstract = presentation.find('div',class_ = 'record__assets ng-star-inserted').find('a')['href']
                presDetailsDict['Abstract Link'] = 'https://meetinglibrary.asco.org/' + abstract

            except:
                pass
            dfDetails = dfDetails.append(presDetailsDict,ignore_index = True)


driver.close()

C:\Users\venug\anaconda3\lib\site-packages\ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


C:\Users\venug\anaconda3\lib\site-packages\ipykernel_launcher.py:37: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


In [68]:
# driver = webdriver.Chrome(r'C:\Users\venug\Downloads\chromedriver.exe')
# dfDetails = pd.DataFrame()
# # for i in tqdm_notebook(SessionLinks[0]):
# presDetailsDict = {}
# driver.get(SessionLinks[0])
# timeout = 500
# locator = '/html/body/div[3]/div/main/div/div/div/ml-root/session/div/div[2]/div/div/div[1]/div[2]/h4'
# WebDriverWait(driver, timeout).until(EC.visibility_of_element_located((By.XPATH, locator)))
# data = driver.page_source
# presList = BeautifulSoup(data,'lxml')
# presentations = presList.find_all('div',class_ = 'session-presentation ng-star-inserted')
# try:
#     sessionMainTitle = presList.find('div',class_ = 'title').text
# except:
#     sessionMainTitle = ''
# try:
#     sessionChairs = presList.find('div',class_ = 'asset-metadata-item session-chair ng-star-inserted').text
# except:
#     sessionChairs = ''
# try:
#     sessionType = presList.find('div',class_ = 'asset-metadata-item session-type ng-star-inserted').text
# except:
#     sessionType = ''
# try:
#     sessionTrack = presList.find('div',class_ = 'asset-metadata-item session-tracks ng-star-inserted').text
# except:
#     sessionTrack = ''
# if(len(presentations) ==0):
#     presDetailsDict = {}
#     presDetailsDict['Session Chairs'] = sessionChairs
#     presDetailsDict['Session Type']  = sessionType
#     presDetailsDict['Session Track'] = sessionTrack
#     presDetailsDict['Session Title'] = sessionMainTitle
#     dfDetails = dfDetails.append(presDetailsDict,ignore_index = True)

# else:
#     for presentation in tqdm_notebook(presentations):
#         presDetailsDict = {}
#         presDetailsDict['Session Chairs'] = sessionChairs
#         presDetailsDict['Session Type']  = sessionType
#         presDetailsDict['Session Track'] = sessionTrack
#         presDetailsDict['Session Title'] = sessionMainTitle
        
#         try:
#             presTitle = presentation.find('div',class_ = 'record__title ng-star-inserted').text
#             presDetailsDict['Presentation Title'] = presTitle
#         except:
#             presTitle = ''
#         details = presentation.findAll('div',class_ = 'record__meta ng-star-inserted')
#         for i in details:
#             try:
#                 presDetailsDict[i.find('div',class_ = 'record__meta__title').text] = i.text
#             except:
#                 pass
#         try:
#             abstract = presentation.find('div',class_ = 'record__assets ng-star-inserted').find('a')['href']
#             presDetailsDict['Abstract Link'] = 'https://meetinglibrary.asco.org/' + abstract
            
#         except:
#             pass
# # #         try:
# # #             presSessTitle = presentation.find('div',class_ = 'presentation-session-title ng-star-inserted').text
# # #             presDetailsDict['Session Title'] = presSessTitle
# # #         except:
# # #             presSessTitle = ''

# #         try:
# #             presChair = presentation.find('div',class_ = 'presentation-role ng-star-inserted').text
# #             presDetailsDict['Chairs'] = presChair 
# #         except:
# #             try:
# #                 presChair = presentation.find('div',class_ = 'author-list ng-star-inserted').text
# #                 presDetailsDict['Chairs'] = presChair 
# #             except:
# #                 presChair = ''


# #         try:
# #             presAbstractDetails = presentation.find('div',class_ = 'abstract-id ng-star-inserted')
# #             try:
# #                 presAbstract = presAbstractDetails.find_all('div',class_ = 'ng-star-inserted')[0].text
# #                 presDetailsDict['Abstract Number'] = presAbstract
# #             except:
# #                 presAbstract = ''
# #             try:
# #                 presPoster = presAbstractDetails.find_all('div',class_ = 'ng-star-inserted')[1].text
# #                 presDetailsDict['Poster Number'] = presPoster
# #             except:
# #                 presPoster = ''
# #         except:
# #             presAbstract = ''
# #             presPoster = ''
# #             pass
#         dfDetails = dfDetails.append(presDetailsDict,ignore_index = True)


# # driver.close()

C:\Users\venug\anaconda3\lib\site-packages\ipykernel_launcher.py:37: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


In [90]:
dfDetails

,,Participant,Presentation Title,Session Chairs,Session Title,Session Track,Session Type,Speaker,Chair,Abstract,Author(s),Abstract Link,Discussant,Panel Speaker,Abstract / Poster
0,,"Participant Clifford A. Hudis, MD, FACP, FASC...","Welcome and Introduction of Howard A. Burris, ...",,Opening Session including Presidential and Gue...,Track(s):Special Sessions,Session Type:Opening Session,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,,NaN,President's Address,,Opening Session including Presidential and Gue...,Track(s):Special Sessions,Session Type:Opening Session,"Speaker Howard A. Burris III, MD, FACP, FASCO...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,,"Participant Howard A. Burris III, MD, FACP, F...","Introduction of David Fajgenbaum, MD, MBA, MSc...",,Opening Session including Presidential and Gue...,Track(s):Special Sessions,Session Type:Opening Session,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,,NaN,Guest Speaker's Address,,Opening Session including Presidential and Gue...,Track(s):Special Sessions,Session Type:Opening Session,"Speaker David C. Fajgenbaum, MD, MBA, MSc, FC...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,,NaN,Chair,"Chair(s):Melissa Lynne Johnson, MD | Sarah Can...",Plenary Session,Track(s):Special Sessions,Session Type:Plenary Session,NaN,"Chair Melissa Lynne Johnson, MD | Sarah Canno...",NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2594,NaN,NaN,Feasibility of a digital medicine program in o...,,Symptoms and Survivorship,Track(s):Symptoms and Survivorship,Session Type:Poster Session,NaN,NaN,NaN,"Author(s) Sherry Shen, Riha Vaidya, Amy Darke...",https://meetinglibrary.asco.org//record/191859...,NaN,NaN,Abstract / Poster TPS12126 ● 414
2595,NaN,NaN,An oro-buccal nanoparticle delivered cannabis ...,,Symptoms and Survivorship,Track(s):Symptoms and Survivorship,Session Type:Poster Session,NaN,NaN,NaN,"Author(s) Stephen John Clarke, Luis Vitetta, ...",https://meetinglibrary.asco.org//record/189572...,NaN,NaN,Abstract / Poster TPS12127 ● 415
2596,NaN,NaN,A phase Ib study of the safety and pharmacolog...,,Symptoms and Survivorship,Track(s):Symptoms and Survivorship,Session Type:Poster Session,NaN,NaN,NaN,"Author(s) Elizabeth J. Adams, Maryam B. Lustb...",https://meetinglibrary.asco.org//record/191850...,NaN,NaN,Abstract / Poster TPS12128 ● 416
2597,NaN,NaN,Use of simulation for training family caregive...,,Symptoms and Survivorship,Track(s):Symptoms and Survivorship,Session Type:Poster Session,NaN,NaN,NaN,"Author(s) Susan R Mazanec, Eric Blackstone, J...",https://meetinglibrary.asco.org//record/191762...,NaN,NaN,Abstract / Poster TPS12129 ● 417


In [91]:
dfDetails.to_excel('ASCO2020.xls')